In [1]:
import mysql.connector

import matplotlib.pyplot as plt
import seaborn as sns
import csv

%matplotlib inline

In [2]:
db_connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="", # change password
  auth_plugin='mysql_native_password'
)
print(db_connection)

In [3]:
db_cursor = db_connection.cursor(buffered=True)

In [4]:
db_cursor.execute("CREATE DATABASE library_database")

In [5]:
db_cursor.execute("USE library_database")

In [6]:
def populate_table(db_connection, db_cursor, insert_query, file_path):
    
    with open(file_path, mode='r') as csv_data:
        reader = csv.reader(csv_data, delimiter=';')
        csv_data_list = list(reader)
        for row in csv_data_list[1:]:
            row = tuple(map(lambda x: None if x == "" else x, row[0].split(',')))
            db_cursor.execute(insert_query, row)
        
    db_connection.commit()

In [7]:
db_cursor.execute("""CREATE TABLE MEMBER (member_ID INTEGER,
                                    fee INTEGER,
                                    fname VARCHAR(50),
                                    lname VARCHAR(50),
                                    PRIMARY KEY (member_ID))""")
insert_member = (
    "INSERT INTO MEMBER(member_ID, fee, fname, lname) "
    "VALUES (%s, %s, %s, %s)"
)
populate_table(db_connection, db_cursor, insert_member, "Member.csv")

In [8]:
db_cursor.execute("""CREATE TABLE Academic_staff (member_ID INTEGER,
                                        PRIMARY KEY (member_ID),
                                        FOREIGN KEY (member_ID) REFERENCES Member(member_ID))""")


In [9]:
insert_academic_staff = (
    "INSERT INTO Academic_staff(member_ID) "
    "VALUES (%s)"
)
populate_table(db_connection, db_cursor, insert_academic_staff, "Academic_staff.csv")

In [10]:
db_cursor.execute("""CREATE TABLE Alumni (member_ID INTEGER,
                                    grad_year INTEGER,
                                    PRIMARY KEY (member_ID),
                                    FOREIGN KEY (member_ID) REFERENCES Member(member_ID))""")

In [11]:
insert_alumni = (
    "INSERT INTO Alumni(member_ID, grad_year) "
    "VALUES (%s, %s)"
)
populate_table(db_connection, db_cursor, insert_alumni, "Alumni.csv")

In [12]:
db_cursor.execute("""CREATE TABLE LIBRARY (library_ID INTEGER,
                                        location VARCHAR(100),
                                        name VARCHAR(10),
                                        PRIMARY KEY (library_id))""")

In [13]:
insert_library = (
    "INSERT INTO Library(library_ID, location, name) "
    "VALUES (%s, %s, %s)"
)
populate_table(db_connection, db_cursor, insert_library, "Library.csv")

In [14]:
db_cursor.execute("""DROP TABLE IF EXISTS `Family_Member`""")

In [15]:

db_cursor.execute("""CREATE TABLE Family_Member (member_ID INTEGER,
                                        fname VARCHAR(30),
                                        lname VARCHAR(30),
                                        PRIMARY KEY (member_ID, fname, lname),
                                        FOREIGN KEY (member_ID) REFERENCES Academic_staff(member_ID))""")

In [16]:
insert_familymember = (
    "INSERT INTO Family_Member(member_ID, fname, lname)"
    "VALUES (%s, %s, %s)"
)

populate_table(db_connection, db_cursor, insert_familymember, "Family_Member.csv")

In [17]:
db_cursor.execute("""DROP TABLE IF EXISTS `Librarian`""")
db_cursor.execute("""CREATE TABLE Librarian (librarian_ID INTEGER,
                                            fname VARCHAR(30),
                                            lname VARCHAR(30),
                                            library_ID INTEGER,
                                            supervisor_ID INTEGER,
                                            start_date DATE,
                                            end_date DATE,
                                            hours INTEGER,
                                            PRIMARY KEY (librarian_ID),
                                            FOREIGN KEY (supervisor_ID) REFERENCES Librarian(librarian_ID),
                                            FOREIGN KEY (library_ID) REFERENCES Library(library_ID))""")


In [18]:
insert_librarian = (
    "INSERT INTO Librarian(librarian_ID, fname, lname, library_ID, supervisor_ID, start_date, end_date, hours)"
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
)
populate_table(db_connection, db_cursor, insert_librarian, "Librarian.csv")

In [19]:
db_cursor.execute("""DROP TABLE IF EXISTS `Student`""")
db_cursor.execute("""CREATE TABLE Student (member_ID INTEGER,
                                            study_level VARCHAR(50),
                                            library_ID INTEGER,
                                            end_date DATE,
                                            start_date DATE,
                                            work_hours INTEGER,
                                            PRIMARY KEY (member_ID),
                                            FOREIGN KEY (member_ID) REFERENCES Member(member_ID),
                                            FOREIGN KEY (library_ID) REFERENCES Library(library_ID))""")

In [20]:
insert_student = (
    "INSERT INTO Student(member_ID, study_level, library_ID, end_date, start_date, work_hours) "
    "VALUES (%s, %s, %s, %s, %s, %s)"
)

populate_table(db_connection, db_cursor, insert_student, "Student.csv")

In [21]:
db_cursor.execute("""DROP TABLE IF EXISTS `Utilities`""")
db_cursor.execute("""CREATE TABLE Utilities (utility_ID INTEGER,
                                                availability VARCHAR(30),
                                                library_ID INTEGER,
                                                PRIMARY KEY (utility_ID),
                                                FOREIGN KEY (library_ID) REFERENCES Library(library_ID))""")
insert_utilities = (
    "INSERT INTO Utilities(utility_ID, availability, library_ID)"
    "VALUES (%s, %s, %s)"
)
populate_table(db_connection, db_cursor, insert_utilities, "Utilities.csv")

In [22]:
db_cursor.execute("""DROP TABLE IF EXISTS `Study_room`""")
db_cursor.execute("""CREATE TABLE Study_room (utility_ID INTEGER,
                                            floor INTEGER,
                                            capacity INTEGER,
                                            tv BOOLEAN,
                                            board BOOLEAN,
                                            PRIMARY KEY (utility_ID),
                                            FOREIGN KEY (utility_ID) REFERENCES Utilities(utility_ID))""")


In [23]:
insert_studyroom = (
    "INSERT INTO Study_room(utility_ID, floor, capacity, tv, board) "
    "VALUES (%s, %s, %s, %s, %s)"
)

populate_table(db_connection, db_cursor, insert_studyroom, "Study_room.csv")

In [24]:
db_cursor.execute("""DROP TABLE IF EXISTS `Printer`""")
db_cursor.execute("""CREATE TABLE Printer (utility_ID INTEGER,
                                                remaining_cartage INTEGER,
                                                PRIMARY KEY (utility_ID),
                                                FOREIGN KEY (utility_ID) REFERENCES Utilities(utility_ID))""")

In [25]:
insert_printer = (
    "INSERT INTO Printer(utility_ID, remaining_cartage)"
    "VALUES (%s, %s)"
)
populate_table(db_connection, db_cursor, insert_printer, "Printer.csv")

In [26]:
db_cursor.execute("""DROP TABLE IF EXISTS `Uses`""")
db_cursor.execute("""CREATE TABLE Uses (s_member_ID INTEGER,
                                        utility_ID INTEGER,
                                        PRIMARY KEY (s_member_ID, utility_ID),
                                        FOREIGN KEY (s_member_ID) REFERENCES Student(member_ID),
                                        FOREIGN KEY (utility_ID) REFERENCES Utilities(utility_ID))""")

In [27]:
insert_uses = (
    "INSERT INTO Uses(s_member_ID, utility_ID)"
    "VALUES (%s, %s)"
)
populate_table(db_connection, db_cursor, insert_uses, "Uses.csv")

In [28]:
db_cursor.execute("""DROP TABLE IF EXISTS `BOOK`""")
db_cursor.execute("""CREATE TABLE BOOK (book_ID INTEGER,
                                        member_ID INTEGER,
                                        library_ID INTEGER,
                                        isbn BIGINT,
                                        title VARCHAR(300),
                                        floor_no INTEGER,
                                        librarian_ID INTEGER,
                                        language VARCHAR(30),
                                        ratings_count INTEGER,
                                        publication_date Date,
                                        publisher VARCHAR(100),
                                        page_number INTEGER, 
                                        rating DOUBLE,
                                        borrow_duration INTEGER,
                                        PRIMARY KEY(book_ID, isbn),
                                        FOREIGN KEY (librarian_ID) REFERENCES Librarian(librarian_ID),
                                        FOREIGN KEY (member_ID) REFERENCES Member(member_ID ),
                                        FOREIGN KEY (library_ID) REFERENCES Library(library_ID))""")


In [29]:
insert_book = (
    "INSERT INTO BOOK(book_ID, member_ID, library_ID, isbn, title, floor_no, librarian_ID, language, ratings_count, publication_date, publisher, page_number, rating, borrow_duration) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

populate_table(db_connection, db_cursor, insert_book, "Books.csv")

In [30]:
db_cursor.execute("""DROP TABLE IF EXISTS `Suggests`""")
db_cursor.execute("""CREATE TABLE Suggests (book_ID INTEGER,
                                            member_ID INTEGER,
                                            FOREIGN KEY (book_ID) REFERENCES Book(book_ID),
                                            FOREIGN KEY (member_ID) REFERENCES Member(member_ID),
                                            PRIMARY KEY (book_ID, member_ID))""")

In [31]:
insert_suggests = (
    "INSERT INTO Suggests(book_ID, member_ID)"
    "VALUES (%s, %s)"
)
populate_table(db_connection, db_cursor, insert_suggests, "Suggests.csv")

In [32]:
db_cursor.execute("""DROP TABLE IF EXISTS `Author`""")
db_cursor.execute("""CREATE TABLE Author (book_ID INTEGER,
                                            authorName VARCHAR(120),
                                            PRIMARY KEY (book_ID, authorName),
                                            FOREIGN KEY (book_ID) REFERENCES Book(book_ID))""")

In [33]:
insert_author = (
    "INSERT INTO Author(book_ID, authorName) "
    "VALUES (%s, %s)"
)
populate_table(db_connection, db_cursor, insert_author, "Author.csv")